In [1]:
from flask import Flask, request
from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api
from json import dumps
from flask_jsonpify import jsonify
import json
import pymongo
import urllib
from collections import Counter
import requests
from flask_swagger import swagger

ImportError: No module named flask_swagger

In [2]:
surveyTemplate = {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'surveyForm':{"0":{
            "qsn":"qsn1",
            "type":"radio",
            "options":["a","b"]
        },
                "1":{
            "qsn":"qsn2",
            "type":"texxt",
            "options":["a","c"]
        }
            
        }
    }

surveyEntry= {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans1"   
        }
    }

In [70]:
app = Flask(__name__)
api = Api(app)

CORS(app)
client = pymongo.MongoClient("mongodb+srv://hemanth:"+ urllib.quote("Hemanth@123")+"@cluster0-59obx.mongodb.net/test?retryWrites=true&w=majority")

@app.route("/spec")
def spec():
    swag = swagger(app)
    swag['info']['version'] = "1.0"
    swag['info']['title'] = "My API"
    return jsonify(swag
                   
@app.route("/")
def hello():
    return jsonify({'text':'Hello World!'})


@app.route("/createSurvey",methods=['POST'])
def createSurvey(): 
    
    verifyAccessToken(request.headers["Authorization"])
    if not(verifyAccessToken):
        return json.dumps({
              "error": "access_denied"
            }), 401
    
    db=client.survey
    params=request.json
    print(params)
    
    if(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName already Exists'})
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["surveyForm"]=params["surveyForm"]
    
    result=db.SurveyForms.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully created'})


@app.route("/surveyForm",methods=['POST'])
def SurveyForm():    
    db=client.survey
    params=request.json
    print(params)
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    result=db.SurveyForms.find_one({'surveyName': params["surveyName"]})
    del result['_id']
    print(result)
    return jsonify(result)


@app.route("/fillSurvey",methods=['POST'])
def SurveyEntry():
    db=client.survey
    params=request.json
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exist'})
    
    surveyDetails={}
    surveyDetails["userName"]=params["userName"]
    surveyDetails["surveyName"]=params["surveyName"]
    surveyDetails["entryForm"]=params["entryForm"]
    
    db.SurveyEntries.insert_one(surveyDetails)
    return  jsonify({'response':'Survey successfully filled'})


# @app.route("/surveystats")
# def SurveyStatistics():
#     db=client.SurveyEntries
#     data=json.loads(str(request.data))
#     if(!checkSurveyExists(request.surveyName)):
#         return jsonify({'response':'SurveyName doesnt Exists'})
    
#     surveyEntries=db.SurveyEntries.find({'surveyName': request.surveyName})
#     surveyTemplate=db.SurveyForms.find({'surveyName': request.surveyName})

#     return surveyEntries

@app.route("/surveystat")
def getByUserSurvey():
    db=client.SurveyEntries
    params=request.json
    if not(checkSurveyExists(params["surveyName"])):
        return jsonify({'response':'SurveyName doesnt Exists'})
    
    surveyEntry=db.SurveyEntries.find({'surveyName': params["surveyName"],'userName':params["userName"]})
    surveyTemplate=db.SurveyForms.find({'surveyName': params["surveyName"]})
    
    return mergeSurventry(surveyTemplate,surveyEntry)

if __name__ == '__main__':
   app.run(port=5002)



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Oct/2019 01:27:26] "OPTIONS /fillSurvey HTTP/1.1" 200 -
127.0.0.1 - - [15/Oct/2019 01:27:28] "POST /fillSurvey HTTP/1.1" 200 -


In [32]:
def checkSurveyExists(surveyName):
    db=client.survey
    form=db.SurveyForms.find_one({'surveyName': surveyName})
    if(form):
        return True
    return False

In [14]:
def mergeSingleSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate["surveyForm"]
    for key in form:
        form[key]["answer"]=surveyEntry["entryForm"][key]
    return form

In [15]:
def mergeSurveyEntry(surveyTemplate,surveyEntry):
    form=surveyTemplate
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=[]
        
    form["users"]=[]
    for entry in surveyEntry:
        form["users"].append(entry["userName"])
        for key in form["surveyForm"]:
            form["surveyForm"][key]["answers"].append(entry["entryForm"][key])
            
    for key in form["surveyForm"]:
        form["surveyForm"][key]["answers"]=Counter(form["surveyForm"][key]["answers"])
        
    return form

In [60]:

def verifyAccessToken(access_token):
    URL="http://127.0.0.1:5001/authorize"
    response=requests.post(url = URL,headers={'Authorization': "access_token"})
    if(response.status_code==200):
        return True
    else:
        return False


In [80]:
surveyTemplate = {
        'userName' : "Hemanth",
        'surveyName' : "test",
        'surveyForm':{"0":{
            "qsn":"qsn1",
            "type":"radio",
            "options":["a","b"]
        },
                "1":{
            "qsn":"qsn2",
            "type":"texxt",
            "options":["a","c"]
        }
            
        }
    }

surveyEntry= [{
        'userName' : "Hemanth",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans1" }
        },{
        'userName' : "Mounika",
        'surveyName' : "test",
        'entryForm':{"0":"ans1",
        "1":"ans12" }
        }]

